In [ ]:
from fsspec.implementations.http import HTTPFileSystem

from emit_tools import emit_xarray
from utils import get_earthdata_token

import pystac_client

# Search the CMR STAC API

This notebook uses the CMR STAC API to search for scenes with a bounding box.
It's trivial to add a datetime range and other parameters to find scenes and
plausible to merge multiple resulting datasets into a temporal stack.

In [ ]:
client = pystac_client.Client.open("https://cmr.earthdata.nasa.gov/cloudstac/LPCLOUD")

# Search with a bbox over Canberra, Australia
results = client.search(collections=["EMITL2ARFL.v001"], bbox=[149.0, -35.5, 149.5, -35.0])

# Print the number of items found
print(f"Found {results.matched()} items")


In [ ]:
for r in results.get_all_items():
  print(f"Found a granule {r.id} from {r.datetime:%Y-%m-%d}")

In [ ]:
# See README.md for instructions on how to get an Earthdata token
token = get_earthdata_token()

In [ ]:
%%time
# Loading data can take around 3-4 minutes on a 100 Mbps connection

# Refer to the README.md for instructions on how to find granule IDs
granule = "EMIT_L2A_RFL_001_20230309T235228_2306815_024"

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + granule + "/" + granule + ".nc"
http_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(http_url))
ds